In [385]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [386]:
BATCH_SIZE = 64
EMBED_SIZE = 384
BLOCK_SIZE = 512
HEAD_SIZE = 6
EPOCHS = 5000
EPOCHS_VAL = 200

In [387]:
with open('./war_and_peace.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [388]:
print(text[:1000])

Лев Николаевич Толстой
Война и мир. Книга 1

Война и мир – 1

Аннотация 

Роман Льва Толстого «Война и мир» лежит в основании величественного здания русской классической литературы. С непревзойденным мастерством Толстой воссоздал великую духом Россию – образы этой «книги на все времена» и сейчас пленяют свежестью чувств и щедростью души, искренностью страстей, силой и чистотой убеждений.
В книгу вошли первый и второй тома романа.

Лев Николаевич Толстой
ВОЙНА И МИР

Том 1

ЧАСТЬ ПЕРВАЯ


I

– Еh bien, mon prince. Genes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous previens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toutes les infamies, toutes les atrocites de cet Antichrist (ma parole, j'y crois) – je ne vous connais plus, vous n'etes plus mon ami, vous n'etes plus мой верный раб, comme vous dites. [Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, 

In [389]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print(''.join(chars))
print(VOCAB_SIZE)


 !&'()*,.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXZ[]`abcdefghijklmnopqrstuvwxyz«»АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё–“„…
146


In [390]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [391]:
print(encode('Привет, как дела?'))
print(decode(encode('Привет, как дела?')))

[94, 125, 117, 111, 114, 127, 8, 1, 119, 109, 119, 1, 113, 114, 120, 109, 22]
Привет, как дела?


In [392]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([1466823])
tensor([ 90, 114, 111,   1,  92, 117, 119, 123, 120, 109, 114, 111, 117, 132,
          1,  97, 123, 120, 126, 127, 123, 118,   0,  81, 123, 118, 122, 109,
          1, 117,   1, 121, 117, 125,   9,   1,  89, 122, 117, 112, 109,   1,
         11,   0,   0,  81, 123, 118, 122, 109,   1, 117,   1, 121, 117, 125,
          1, 142,   1,  11,   0,   0,  79, 122, 122, 123, 127, 109, 131, 117,
        140,   1,   0,   0,  95, 123, 121, 109, 122,   1,  90, 137, 111, 109,
          1,  97, 123, 120, 126, 127, 123, 112, 123,   1,  77,  81, 123, 118,
        122, 109,   1, 117,   1, 121, 117, 125,  78,   1, 120, 114, 115, 117,
        127,   1, 111,   1, 123, 126, 122, 123, 111, 109, 122, 117, 117,   1,
        111, 114, 120, 117, 132, 114, 126, 127, 111, 114, 122, 122, 123, 112,
        123,   1, 116, 113, 109, 122, 117, 140,   1, 125, 128, 126, 126, 119,
        123, 118,   1, 119, 120, 109, 126, 126, 117, 132, 114, 126, 119, 123,
        118,   1, 120, 117, 127, 114, 125,

In [393]:
split_idx = int(0.9*len(data))
train_data = data[:split_idx]
test_data = data[split_idx:]

In [394]:
block_size = 8
train_data[:block_size+1]

tensor([ 90, 114, 111,   1,  92, 117, 119, 123, 120])

In [395]:
def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (BATCH_SIZE,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print('Input:')
print(xb)

print('Target:')
print(yb)



Input:
tensor([[ 57,  71,  55,  68,  68,  55,   8,   1],
        [140,   1, 126,   1, 126, 111, 123, 117],
        [137,   1, 124, 114, 126, 122, 117,   8],
        [112, 123, 118,   1, 126,   1, 113, 114],
        [133, 114, 122, 122, 136, 121,   1, 124],
        [109, 111, 133, 109, 140, 126, 140,   1],
        [116, 122, 109, 122, 117, 114,   8,   1],
        [116, 140, 111,   1, 117, 116,   1, 133],
        [113, 128, 139, 134, 114, 112, 123,   9],
        [123,   1, 110, 136, 120, 123,   1, 114],
        [109, 139,   8,   1, 109,   1, 127, 114],
        [140, 120, 117,   1, 124, 125, 117, 126],
        [  1, 116, 128, 110, 136,   8,   1, 133],
        [114, 125, 109, 120,   8,   1, 132, 127],
        [  8,   1, 119, 127, 123,   1, 120, 123],
        [112, 123,   1, 113, 123, 110, 125, 109],
        [128, 120, 136, 110, 119, 109,   9,   0],
        [123, 126, 127, 137, 139,   9,   1,  93],
        [109,   1, 126, 111, 123, 117, 121,   1],
        [113, 117, 120, 126, 140,  21,   1,

In [396]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EPOCHS_VAL)
        for k in range(EPOCHS_VAL):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [397]:
class Head(nn.Module):
    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.query = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.value = nn.Linear(EMBED_SIZE, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        w = q @ k.transpose(1, 2) * C**-0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)
        v = self.value(x)
        out = w @ v
        return out

In [398]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(EMBED_SIZE, EMBED_SIZE)
        self.dropout = nn.Dropout(0.2)

    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))

In [399]:
class FFN(nn.Module):
    def __init__(self, embed_size):
        super(FFN, self).__init__()
        self.ffn = nn.Sequential(
            nn.Linear(embed_size, embed_size*4),
            nn.ReLU(),
            nn.Linear(embed_size*4, embed_size),
            nn.Dropout(0.2)
        )

    
    def forward(self, x):
        return self.ffn(x)

In [400]:
class Block(nn.Module):
    def __init__(self, embed_size, head_size):
        super(Block, self).__init__()
        h_size = embed_size // head_size
        self.sa = MultiHeadAttention(head_size, h_size)
        self.ffn = FFN(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return(x)

In [401]:
class LM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embed = nn.Embedding(VOCAB_SIZE, EMBED_SIZE)
        self.pos_embed = nn.Embedding(BLOCK_SIZE, EMBED_SIZE)
        self.blocks = nn.Sequential(*[Block(EMBED_SIZE, HEAD_SIZE) for _ in range(6)])
        self.ln = nn.LayerNorm(EMBED_SIZE)
        self.lm_head = nn.Linear(EMBED_SIZE, VOCAB_SIZE)
        

    def forward(self, x, y=None):
        B, T = x.shape
        tok_embed = self.token_embed(x)
        pos_embed = self.pos_embed(torch.arange(T))
        out = tok_embed + pos_embed
        out = self.blocks(out)
        out = self.ln(out)
        out = self.lm_head(out)
        
        if y is None:
            loss = None
        else:
            B, T, C = out.shape
            out = out.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(out, y)

        return out, loss


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) 

    
    def generate(self, x, max_new_tokens):
        for _ in range(max_new_tokens):
            x_cond = x[:, -block_size:]
            out, loss = self(x_cond)
            out = out[:, -1, :]
            probs = F.softmax(out, dim=-1)
            x_next = torch.multinomial(probs, num_samples=1)
            x = torch.cat((x, x_next), dim=1)
        return x

In [402]:
model = LM()
out, loss = model(xb, yb)
print(out.shape)
print(loss.item())

torch.Size([512, 146])
5.212309837341309


In [403]:
best_model = None
best_loss = float('inf')

In [404]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for epoch in range(EPOCHS):
    xb, yb = get_batch('train')

    out, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if losses['val'] < best_loss:
            best_loss = losses['val']
            best_model = model.state_dict()

step 0: train loss 4.1682, val loss 4.1386
step 500: train loss 2.2335, val loss 2.2131
step 1000: train loss 2.0937, val loss 2.0724
step 1500: train loss 2.0310, val loss 2.0091
step 2000: train loss 1.9790, val loss 1.9622
step 2500: train loss 1.9444, val loss 1.9242
step 3000: train loss 1.9184, val loss 1.9079
step 3500: train loss 1.8948, val loss 1.8877
step 4000: train loss 1.8716, val loss 1.8727
step 4500: train loss 1.8704, val loss 1.8435


In [409]:
x = torch.zeros((1, 1), dtype=torch.long)
generated_text = decode(model.generate(x, max_new_tokens=10000)[0].tolist())
generated_text

"\n«Анна Памений у возвратеин правлость в середнееру моя чомным, офицанить еще унибилье последных била тешать еёвого воздорого, шапь на приближение тоже этом его вам не полкомый положила лошадь все с.\nИму 1ста читой, поль был,! – сказал государом положения да ИоНоБочных и Лужиках, даже этом делает ему были людовию в домой время, и вывел в когрому, хочется. Князь Андрей. Но маторовник свое увторово знакочить пролки, я взялся этому гляников на мысли уподал крики, пресколь? Сонюшка – Пусгой на ликелуший на была на обм0 разна нает. Княнным и для свое и влеон у ион с седне брестящурь зсорил радоровая в ценелим» этого Соняющие, больь себя частью, чтобы время волчал пододвижно. Причастия военна при деля, что он на не имел на Тишка, что он в знаешего подъезд к да. За читальном что он желая, французский при игратился Пьiра; но воюсь с дочерой стерила радоставляленно ступложил Пьер Успитался, то себя, чтобы соботама?\n– Я просавлюем похожье? – говорили здесьмо, рукой.\n– Ну, моя выхороном, и бр

In [410]:
with open('generated_text.txt', 'w', encoding='utf-8') as f:
    f.write(generated_text)

In [406]:
torch.save(model.state_dict(), 'war_and_peace_transformer.pth')